### Capstone Project

In [4]:
import os 
import pandas as pd
import plotly.express as plt
from datetime import timedelta
from datetime import datetime

### Step-1: Load Data by Path

In [5]:
Root_path = 'C:\\Users\\mdmes\\OneDrive\\Desktop\\Pandas Data'

product_path = os.path.join(Root_path,'products.csv')
purchases_path =os.path.join(Root_path, 'purchases.csv')
sales_path = os.path.join(Root_path, 'sales.csv')


In [6]:
product_data = pd.read_csv(
    product_path
)
product_data.head()

,product_id,product_name,category,cost_price,selling_price,reorder_level
0,1001,Organic Rice,Groceries,1551.26,2296.31,43
1,1002,Olive Oil,Groceries,2516.38,3370.60,39
2,1003,Canned Beans,Groceries,961.25,1117.36,30
3,1004,Pasta,Groceries,696.83,1007.94,36
4,1005,Coffee Beans,Groceries,2163.01,2991.94,27


In [8]:
purchases_data = pd.read_csv(
    purchases_path
)

purchases_data.head()

,product_id,quantity_purchased,purchase_date
0,1095,94,2024-08-19
1,1054,23,2024-08-16
2,1058,64,2024-12-18
3,1067,57,2024-06-12
4,1046,91,2024-03-13


In [9]:
sales_data = pd.read_csv(
    sales_path
)

sales_data.head()

,product_id,quantity_sold,sale_date,location,discount
0,1013,4,2024-09-02,Rajshahi,52.178882
1,1064,9,2024-02-25,Sylhet,52.059982
2,1011,2,2024-02-07,Dhaka,7.370982
3,1019,2,2024-09-01,Sylhet,51.219448
4,1011,9,2024-01-21,Chittagong,82.274403


### Step-2:Convert string to datetime 

In [ ]:
sales_data['sale_date'] = pd.to_datetime(sales_data['sale_date']).dt.date